In [1]:
!pip install datasets transformers evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# CARGA DE DATASET

In [2]:
from datasets import load_dataset

ds = load_dataset("tyqiangz/multilingual-sentiments", "spanish")
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

multilingual-sentiments.py:   0%|          | 0.00/6.23k [00:00<?, ?B/s]

spanish/train/0000.parquet:   0%|          | 0.00/120k [00:00<?, ?B/s]

spanish/validation/0000.parquet:   0%|          | 0.00/24.1k [00:00<?, ?B/s]

spanish/test/0000.parquet:   0%|          | 0.00/58.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1839 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/324 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/870 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 1839
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 324
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 870
    })
})

In [3]:
ex = ds["train"][100]
ex

{'text': 'Después de siete largos años vuelvo a pasar navidad junto con mi mamá, mi papá y mi hermana. Todos juntos con una yapita de sobrino.',
 'source': 'intertass_2017',
 'label': 1}

In [4]:
labels = ds["train"].features["label"]
labels

ClassLabel(names=['positive', 'neutral', 'negative'], id=None)

# TOKENIZACION

In [5]:
from transformers import AutoTokenizer

repo_id = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(repo_id)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [6]:
tokenized_sentence_1 = tokenizer(ds['train']['text'][2])
tokenized_sentence_1

{'input_ids': [101, 24165, 10104, 16627, 100898, 39520, 10113, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
def tokenize_fn(texts):
  return tokenizer(texts['text'],texts['source'],truncation=True)

In [22]:
prepared_ds = ds.map(tokenize_fn,batched=True)
prepared_ds

Map:   0%|          | 0/1839 [00:00<?, ? examples/s]

Map:   0%|          | 0/324 [00:00<?, ? examples/s]

Map:   0%|          | 0/870 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1839
    })
    validation: Dataset({
        features: ['text', 'source', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 324
    })
    test: Dataset({
        features: ['text', 'source', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 870
    })
})

# PADDING

In [7]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# METRICAS

In [10]:
#se usara una nueva metrica
import evaluate
import numpy as np


def compute_metrics(eval_pred):
  #metric = evaluate.load("tyqiangz/multilingual-sentiments", "spanish")
  metric = evaluate.load("accuracy") # se cambia la metrica
  logits,labels = eval_pred
  predictions = np.argmax(logits,axis=-1)
  return metric.compute(predictions=predictions,references=labels)


## CONFIGURACION DEL MODELO

In [11]:
from transformers import AutoModelForSequenceClassification

labels = ds['train'].features['label'].names

model = AutoModelForSequenceClassification.from_pretrained(
    repo_id,
    num_labels=len(labels),
    id2label={str(i): c for i,c in enumerate(labels)},
    label2id={c: str(i) for i,c in enumerate(labels)}
)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir ='./csanchezcsdigitales-distilroberta-base-mrpc-glue-csanchezc',
    hub_model_id='csanchezcsdigitales/csanchezcsdigitales-distilroberta-base-mrpc-glue-csanchezc',
    evaluation_strategy='epoch',
    num_train_epochs=3,
    push_to_hub=True,
    save_strategy='epoch',
    load_best_model_at_end=True,
    weight_decay = 0.01,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    learning_rate = 2e-5,
    metric_for_best_model = 'accuracy',
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `cesar_token1` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenti

In [27]:
from transformers import Trainer

trainer = Trainer(
  model,
  training_args,
  train_dataset=prepared_ds['train'],
  eval_dataset=prepared_ds['validation'],
  data_collator=data_collator,
  tokenizer=tokenizer,
  compute_metrics=compute_metrics
)


<ipython-input-27-d51073d4ec85>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [28]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train",train_results.metrics)
trainer.save_metrics("train",train_results.metrics)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.974396,0.509259
2,No log,0.881610,0.586420
3,No log,0.867524,0.614198


events.out.tfevents.1732229278.f3f93b32686d.1444.0:   0%|          | 0.00/6.91k [00:00<?, ?B/s]

***** train metrics *****
  epoch                    =        3.0
  total_flos               =   126428GF
  train_loss               =     0.9147
  train_runtime            = 0:10:48.04
  train_samples_per_second =      8.513
  train_steps_per_second   =      0.532


##evaluacion de metricas

In [29]:
metrics = trainer.evaluate(prepared_ds["validation"])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.6142
  eval_loss               =     0.8675
  eval_runtime            = 0:00:02.21
  eval_samples_per_second =    146.338
  eval_steps_per_second   =      9.485
